In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,45370
2,Huelva,Confirmados PDIA 14 días,303
3,Huelva,Tasa PDIA 14 días,"59,044760995381644"
4,Huelva,Confirmados PDIA 7 días,107
5,Huelva,Tasa PDIA 7 dias,"20,85079018648791"
6,Huelva,Total Confirmados,45573
7,Huelva,Curados,44311
8,Huelva,Fallecidos,415


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  45370.0


/tmp/ipykernel_4273/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12537.0


/tmp/ipykernel_4273/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_4273/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_4273/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_4273/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 6254 personas en los últimos 7 días 

Un positivo PCR cada 2147 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,45370.0,107.0,303.0,513170.0,20.850790,59.044761,30.0
Huelva-Costa,26861.0,68.0,182.0,289548.0,23.484880,62.856590,18.0
Condado-Campiña,14082.0,29.0,104.0,156231.0,18.562257,66.568095,10.0
Huelva (capital),12537.0,23.0,67.0,143837.0,15.990322,46.580504,9.0
Cartaya,1974.0,14.0,25.0,20083.0,69.710701,124.483394,4.0
Moguer,1956.0,5.0,17.0,21867.0,22.865505,77.742717,3.0
Palos de la Frontera,1030.0,4.0,17.0,11742.0,34.065747,144.779424,3.0
Bollullos Par del Condado,1282.0,6.0,21.0,14387.0,41.704316,145.965107,2.0
Isla Cristina,3125.0,13.0,21.0,21393.0,60.767541,98.162950,2.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Campofrío,22.0,2.0,2.0,713.0,280.504909,280.504909,NaN
Rosal de la Frontera,74.0,3.0,3.0,1697.0,176.782557,176.782557,0.0
Bollullos Par del Condado,1282.0,6.0,21.0,14387.0,41.704316,145.965107,2.0
Palos de la Frontera,1030.0,4.0,17.0,11742.0,34.065747,144.779424,3.0
Ayamonte,2071.0,4.0,28.0,21104.0,18.953753,132.676270,1.0
Cartaya,1974.0,14.0,25.0,20083.0,69.710701,124.483394,4.0
Gibraleón,936.0,6.0,13.0,12737.0,47.106854,102.064850,0.0
Isla Cristina,3125.0,13.0,21.0,21393.0,60.767541,98.162950,2.0
Cerro de Andévalo (El),198.0,1.0,2.0,2327.0,42.973786,85.947572,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Ayamonte,2071.0,4.0,28.0,21104.0,18.953753,132.676270,1.0,0.142857
Almonte,2285.0,4.0,21.0,24507.0,16.321867,85.689803,1.0,0.190476
Palos de la Frontera,1030.0,4.0,17.0,11742.0,34.065747,144.779424,3.0,0.235294
Lepe,3008.0,5.0,18.0,27880.0,17.934003,64.562410,1.0,0.277778
Condado-Campiña,14082.0,29.0,104.0,156231.0,18.562257,66.568095,10.0,0.278846
Bollullos Par del Condado,1282.0,6.0,21.0,14387.0,41.704316,145.965107,2.0,0.285714
Moguer,1956.0,5.0,17.0,21867.0,22.865505,77.742717,3.0,0.294118
Huelva (capital),12537.0,23.0,67.0,143837.0,15.990322,46.580504,9.0,0.343284
Huelva,45370.0,107.0,303.0,513170.0,20.850790,59.044761,30.0,0.353135
